In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append('../')

In [3]:
import os
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from utilities import save_model as save

home_dir = os.path.join(os.path.expanduser('~'), 'Documents/TDT4173-ML')
train_data_path = os.path.join(home_dir, 'data_feature_engineering')
target_data_path = os.path.join(home_dir, 'data_processed')

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321.

.... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyyhb7e8l
  JVM stdout: /tmp/tmpyyhb7e8l/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmpyyhb7e8l/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_stinky_iqva11
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [5]:
train = pd.read_csv(os.path.join(train_data_path, 'summed_no_snow_train.csv'))
target = pd.read_csv(os.path.join(target_data_path, 'target.csv'))

train.set_index('date_forecast', inplace=True)
target.set_index('time', inplace=True)

In [6]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
autoML = H2OAutoML(max_models=30, seed=6)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |
11:45:55.971: XGBoost_1_AutoML_1_20231102_114555 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_1_20231102_114555_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


██████████████████
11:48:12.990: XGBoost_2_AutoML_1_20231102_114555 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_1_20231102_114555_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█████████████████████████
11:56:52.506: XGBoost_3_AutoML_1_20231102_114555 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_1_20231102_114555_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NA

In [ ]:
best_model = autoML.leader
save.save_model_to_local_disk(best_model, '01E_h2o_summed_no_snow')